In [19]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas 

pn.extension()

In [7]:
# cache data for performance improvement 

if 'data' not in pn.state.cache.keys():
    data = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')
    pn.state.cache['data'] = data.copy()
    
else:
    data = pn.state.cache['data']
    
data.head(3)

,iso_code,country,year,co2,co2_per_capita,trade_co2,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,...,ghg_excluding_lucf_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,0.002,NaN,NaN,NaN,0.015,0.002,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,0.011,NaN,NaN,NaN,0.021,0.003,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,0.012,NaN,NaN,NaN,0.026,0.003,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN


In [8]:
# minor data preprocessing

data = data.fillna(0)
data['gdp_per_capita'] = np.where(data['population'] != 0,
                                  data['gdp'] / data['population'],
                                  0)


In [22]:
# make dataframe pipeline interactive 
idf = data.interactive()

### CO2 emission over time by continent 

In [20]:
# define panel widgets 

year_slider = pn.widgets.IntSlider(name='Year slider', 
                                   start=1750, 
                                   end=2020,
                                   step=5, 
                                   value=1850)
year_slider

IntSlider(end=2020, name='Year slider', start=1750, step=5, value=1850)

In [11]:
# radio buttons for co2 measures 

yaxis_co2 = pn.widgets.RadioButtonGroup(
            name='Y-axis',
            options=['co2', 'co2_per_capita'],
            button_type='success'
)

In [24]:
STEP = 5

continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 
              'North America', 'South America', 'Antarctica']

co2_pipeline = (
        idf[
            (idf.year == year_slider) &
            (idf.country.isin(continents))
        ]
        .groupby(['country', 'year'])[yaxis_co2].mean()
        .to_frame()
        .reset_index()
        .sort_values(by='year')
        .reset_index(drop=True)
)

co2_pipeline

### Table - CO2 emission over time by continent


In [25]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, 
                              pagination='remote',
                              page_size=10, 
                              sizing_mode='stretch_width')
co2_table

### CO2 vs GDP scatterplot


In [28]:
co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [29]:
co2_vs_gdp_scatterplot_pipeline

In [30]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(
        x = 'gdp_per_capita',
        by='country',
        size=80,
        kind='scatter',
        alpha=0.7,
        legend=False,
        height=500,
        width=500
)

co2_vs_gdp_scatterplot